In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs

In [3]:
path = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ACR_14/hypnograms/hypno_swi-bl_chunk1_short-art.txt'
h = kh.load_hypno_file(path, st=None, dt=False)

In [5]:
h.to_csv('/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ACR_14/hypnograms/ss_test_short.csv')